In [ ]:
%%writefile finetune_llava_job.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json
code: ./src

command: >- 
  echo "./finetune_llava.sh ${{inputs.data_dir}} ${{outputs.out_dir}}"; sleep 60000
inputs:
  data_dir: 
    type: uri_folder
    mode: ro_mount
    #path: azureml:Multimodal-Fatima_OK@latest
outputs:
  out_dir: 
    type: uri_folder
    mode: upload
environment: azureml:llava_finetuning@latest
resources:
  instance_count: 1
distribution:
  type: pytorch 
  process_count_per_instance: 4
services:
    my_vs_code:
      type: vs_code
      nodes: all # For distributed jobs, use the `nodes` property to pick which node you want to enable interactive services on. If `nodes` are not selected, by default, interactive applications are only enabled on the head node. Values are "all", or compute node index (for ex. "0", "1" etc.)
    my_jupyter_lab:
      type: jupyter_lab
      nodes: all
#compute: azureml:fine-tune-cluster
compute: azureml:a100-low-priority
display_name: finetune_llava_job
name: finetune_llava_job
experiment_name: finetune_llava
description: Finetune LlaVa

In [ ]:
!az ml job create -f ./finetune_llava_job.yml

In [ ]:
!az ml component create -f ./finetune_llava_job.yml

## Merge the Finetuned weights

In [1]:
%%writefile merge_finetuned_llava_job.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json
command: >- 
  python /LLaVA/scripts/merge_lora_weights.py --model-path ${{inputs.checkpoints_home}}/checkpoints/llava-v1.5-13b-task-lora --model-base liuhaotian/llava-v1.5-13b --save-model-path ${{outputs.llava_out_dir}}
inputs:
  checkpoints_home: 
    type: uri_folder
    #mode: ro_mount
    path: azureml:azureml_bright_office_tsywj86mp0_output_data_out_dir:1
outputs:
  llava_out_dir: 
    type: custom_model
    mode: upload
environment: azureml:llava_finetuning:16
resources:
  instance_count: 1
distribution:
  type: pytorch 
  process_count_per_instance: 4
services:
    my_vs_code:
      type: vs_code
      nodes: all # For distributed jobs, use the `nodes` property to pick which node you want to enable interactive services on. If `nodes` are not selected, by default, interactive applications are only enabled on the head node. Values are "all", or compute node index (for ex. "0", "1" etc.)
    my_jupyter_lab:
      type: jupyter_lab
      nodes: all
compute: azureml:antonslutsky1
#compute: azureml:fine-tune-cluster
#compute: azureml:a100-low-priority
display_name: merge_finetuned_llava_job
#name: merge_finetuned_llava_job
experiment_name: finetune_llava
description: Save finetuned LlaVa weights.

Overwriting merge_finetuned_llava_job.yml


In [2]:
!az ml job create -f ./merge_finetuned_llava_job.yml

{
  "command": "python /LLaVA/scripts/merge_lora_weights.py --model-path ${{inputs.checkpoints_home}}/checkpoints/llava-v1.5-13b-task-lora --model-base liuhaotian/llava-v1.5-13b --save-model-path ${{outputs.llava_out_dir}}",
  "compute": "azureml:antonslutsky1",
  "creation_context": {
    "created_at": "2024-05-31T13:30:03.398256+00:00",
    "created_by": "Anton Slutsky",
    "created_by_type": "User"
  },
  "description": "Save finetuned LlaVa weights.",
  "display_name": "merge_finetuned_llava_job",
  "distribution": {
    "process_count_per_instance": 4,
    "type": "pytorch"
  },
  "environment": "azureml:llava_finetuning:16",
  "environment_variables": {},
  "experiment_name": "finetune_llava",
  "id": "azureml:/subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourceGroups/antonslutsky-rg/providers/Microsoft.MachineLearningServices/workspaces/gpu-workspace/jobs/tough_plow_b5dv514mdv",
  "inputs": {
    "checkpoints_home": {
      "mode": "ro_mount",
      "path": "azureml:azu

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [ ]:
!az ml component create -f ./merge_finetuned_llava_job.yml

## Apply the model

In [ ]:
%%writefile apply_finetuned_llava_job.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json
command: >- 
  echo "Model Home: ${{inputs.llava_model_home}} Output Dir: ${{outputs.out_dir}}"; sleep 70000
inputs:
  llava_model_home: 
    type: uri_folder
    #mode: ro_mount
    path: azureml:azureml_tough_cherry_s5gzbchvct_output_data_out_dir:1
outputs:
  out_dir: 
    type: uri_folder
    mode: upload
environment: azureml:llava_finetuning@latest
resources:
  instance_count: 1
distribution:
  type: pytorch 
  process_count_per_instance: 4
services:
    my_vs_code:
      type: vs_code
      nodes: all # For distributed jobs, use the `nodes` property to pick which node you want to enable interactive services on. If `nodes` are not selected, by default, interactive applications are only enabled on the head node. Values are "all", or compute node index (for ex. "0", "1" etc.)
    my_jupyter_lab:
      type: jupyter_lab
      nodes: all
compute: azureml:fine-tune-cluster
#compute: azureml:a100-low-priority
display_name: apply_finetuned_llava_job
experiment_name: finetune_llava
#name: apply_finetuned_llava_job
description: Apply finetuned LlaVa weights.

In [ ]:
!az ml job create -f ./apply_finetuned_llava_job.yml

In [ ]:
!az ml component create -f ./apply_finetuned_llava_job.yml